In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

customer_df = pd.read_csv("/Users/suso/BootcampDA/Unit4/Labs/lab-cleaning-categorical-data/files_for_lab/we_fn_use_c_marketing_customer_value_analysis.csv")
new_column_names = {
    'Customer': 'customer',
    'State': 'state',
    'Customer Lifetime Value': 'customer_lifetime_value',
    'Response': 'response',
    'Coverage': 'coverage',
    'Education': 'education',
    'Effective To Date': 'effective_to_date',
    'EmploymentStatus': 'employment_status',
    'Gender': 'gender',
    'Income': 'income',
    'Location Code': 'location_code',
    'Marital Status': 'marital_status',
    'Monthly Premium Auto': 'monthly_premium_auto',
    'Months Since Last Claim': 'months_since_last_claim',
    'Months Since Policy Inception': 'months_since_policy_inception',
    'Number of Open Complaints': 'number_of_open_complaints',
    'Number of Policies': 'number_of_policies',
    'Policy Type': 'policy_type',
    'Policy': 'policy',
    'Renew Offer Type': 'renew_offer_type',
    'Sales Channel': 'sales_channel',
    'Total Claim Amount': 'total_claim_amount',
    'Vehicle Class': 'vehicle_class',
    'Vehicle Size': 'vehicle_size',
}

customer_df = customer_df.rename(columns=new_column_names)

customer_df['effective_to_date'] = pd.to_datetime(customer_df['effective_to_date'])

customer_df = customer_df.drop('customer', axis=1)

numerical_df = customer_df.select_dtypes(include=['int64', 'float64'])
categorical_df = customer_df.select_dtypes(include=['object'])

In [3]:
#1. We will start with removing outliers. So far, we have discussed different methods to remove outliers. Use the one you feel more comfortable with, define a function for that. Use the function to remove the outliers and apply it to the dataframe.

import numpy as np

def remove_outliers_zscore(customer_df, numerical_df, threshold=3):
    z_scores = np.abs((numerical_df - numerical_df.mean()) / numerical_df.std())
    outliers = (z_scores > threshold).any(axis=1)
    return customer_df[~outliers]

cleaned_df = remove_outliers_zscore(customer_df, numerical_df)

In [5]:
#2. Create a copy of the dataframe for the data wrangling.

cleaned_df_copy = cleaned_df.copy()

In [7]:
#3. Normalize the continuous variables. You can use any one method you want.

from sklearn.preprocessing import MinMaxScaler

numericals = cleaned_df_copy.select_dtypes(include=['int64', 'float64'])
categoricals = cleaned_df_copy.select_dtypes(include=['object'])

scaler = MinMaxScaler()
numericals = scaler.fit_transform(numericals)

In [20]:
unique_values_multiple = categoricals.apply(lambda x: x.unique())
print(unique_values_multiple)
unique_values_variable1 = cleaned_df_copy['vehicle_class'].unique()
print(unique_values_variable1)


state                [Washington, Arizona, Nevada, California, Oregon]
response                                                     [No, Yes]
coverage                                    [Basic, Extended, Premium]
education            [Bachelor, College, Master, High School or Bel...
employment_status    [Employed, Unemployed, Medical Leave, Disabled...
gender                                                          [F, M]
location_code                                 [Suburban, Rural, Urban]
marital_status                             [Married, Single, Divorced]
policy_type              [Corporate Auto, Personal Auto, Special Auto]
policy               [Corporate L3, Personal L3, Corporate L2, Pers...
renew_offer_type                      [Offer1, Offer3, Offer2, Offer4]
sales_channel                        [Agent, Call Center, Web, Branch]
vehicle_class        [Two-Door Car, Four-Door Car, SUV, Sports Car,...
vehicle_size                                   [Medsize, Small, Large]
dtype:

In [21]:
#4. Encode the categorical variables

cleaned_df_copy["coverage"] = cleaned_df_copy["coverage"].map({"Basic": 0, "Extended": 1, "Premium": 2})
cleaned_df_copy["state"] = cleaned_df_copy["state"].map({"Washington": 0, "Arizona": 1, "Nevada": 2, "California": 3, "Oregon": 4})
cleaned_df_copy["response"] = cleaned_df_copy["response"].map({"No": 0, "Yes": 1})
cleaned_df_copy["education"] = cleaned_df_copy["education"].map({"Bachelor": 0, "College": 1, "Master": 2, "High School or Below": 3, "Doctor": 4})
cleaned_df_copy["employment_status"] = cleaned_df_copy["employment_status"].map({"Employed": 0, "Unemployed": 1, "Medical Leave": 2, "Disabled": 3, "Retired": 4})
cleaned_df_copy["gender"] = cleaned_df_copy["gender"].map({"F": 0, "M": 1})
cleaned_df_copy["location_code"] = cleaned_df_copy["location_code"].map({"Suburban": 0, "Rural": 1, "Urban": 2})
cleaned_df_copy["marital_status"] = cleaned_df_copy["marital_status"].map({"Married": 0, "Single": 1, "Divorced": 2})
cleaned_df_copy["policy_type"] = cleaned_df_copy["policy_type"].map({"Corporate Auto": 0, "Personal Auto": 1, "Special Auto": 2})
cleaned_df_copy["policy"] = cleaned_df_copy["policy"].map({"Corporate L3": 0, "Personal L3": 1, "Corporate L2": 2, "Personal L1": 3, "Special L2": 4, "Corporate L1": 5, "Personal L2": 6, "Special L1": 7, "Special L3": 8})
cleaned_df_copy["renew_offer_type"] = cleaned_df_copy["renew_offer_type"].map({"Offer1": 0, "Offer3": 1, "Offer2": 2, "Offer4": 3})
cleaned_df_copy["sales_channel"] = cleaned_df_copy["sales_channel"].map({"Agent": 0, "Call Center": 1, "Web": 2, "Branch": 3})
cleaned_df_copy["vehicle_class"] = cleaned_df_copy["vehicle_class"].map({"Two-Door Car": 0, "Four-Door Car": 1, "SUV": 2, "Sports Car": 3, "Luxury SUV": 4, "Luxury Car": 5})
cleaned_df_copy["vehicle_size"] = cleaned_df_copy["vehicle_size"].map({"Medsize": 0, "Small": 1, "Large": 2})

In [10]:
#5. The time variable can be useful. Try to transform its data into a useful one. Hint: Day week and month as integers might be useful.

cleaned_df_copy['day_of_week'] = cleaned_df_copy['effective_to_date'].dt.dayofweek
cleaned_df_copy['month'] = cleaned_df_copy['effective_to_date'].dt.month

In [22]:
#6. Since the model will only accept numerical data, check and make sure that every column is numerical, if some are not, change it using encoding.

non_numeric_columns = []
for column in cleaned_df_copy.columns:
    if not pd.api.types.is_numeric_dtype(cleaned_df_copy[column]):
        non_numeric_columns.append(column)

In [23]:
print(non_numeric_columns)

['effective_to_date']


In [24]:
cleaned_df_copy = cleaned_df_copy.drop('effective_to_date', axis=1)

In [25]:
non_numeric_columns = []
for column in cleaned_df_copy.columns:
    if not pd.api.types.is_numeric_dtype(cleaned_df_copy[column]):
        non_numeric_columns.append(column)

In [26]:
print(non_numeric_columns)

[]
